In [30]:
import classiq

from classiq.builtin_functions import StatePreparation
from classiq import RegisterUserInput
from classiq.builtin_functions import Adder
from classiq import Model

# for execution
from classiq.execution import ExecutionPreferences, IBMBackendPreferences
from classiq.synthesis import set_execution_preferences

# for synthesis
from classiq.model import Preferences
from classiq.synthesis import set_preferences

#for optimisation
from classiq.model import Constraints
from classiq.synthesis import set_constraints

from classiq import synthesize

# visualise
from classiq import GeneratedCircuit
from classiq import show

# execution on device
from classiq import execute
from classiq.execution import ExecutionDetails


In [24]:
classiq.authenticate()

Your user code: VLDH-SBVL
If a browser doesn't automatically open, please visit the url: https://auth.classiq.io/activate?user_code=VLDH-SBVL


In [5]:
prob_a = [0.5, 0, 0, 0.5]
prob_b = [0, 0.5, 0.5, 0]

In [6]:
sp_a = StatePreparation(
    probabilities=prob_a, error_metric={"KL": {"upper_bound": 0.01}}
)

sp_b = StatePreparation(
    probabilities=prob_b, error_metric={"KL": {"upper_bound": 0.01}}
)

In [7]:

adder = Adder(
    left_arg=RegisterUserInput(size=2),
    right_arg=RegisterUserInput(size=2),
)

In [8]:
model = Model()


In [9]:
a = model.StatePreparation(params=sp_a)
b = model.StatePreparation(params=sp_b)

In [10]:
adder_out = model.Adder(
    params=adder, in_wires={"left_arg": a["OUT"], "right_arg": b["OUT"]}
)

In [11]:
model.set_outputs(
    {"a": adder_out["left_arg"], "b": adder_out["right_arg"], "sum": adder_out["sum"]}
)
# sum is added here to tell the model this is what we are interested in

In [12]:
model.sample()

In [14]:
backend_preferences = IBMBackendPreferences(
    backend_service_provider="IBM Quantum", backend_name="aer_simulator_statevector"
)

serialized_model = model.get_model()

serialized_model = set_execution_preferences(
    serialized_model,
    execution_preferences=ExecutionPreferences(backend_preferences=backend_preferences),
)

In [16]:
seed = 206755496
preferences = Preferences(random_seed=seed)
serialized_model = set_preferences(serialized_model, preferences=preferences)

In [18]:

constraints = Constraints(optimization_parameter="depth")
serialized_model = set_constraints(serialized_model, constraints=constraints)

In [ ]:
qmod = model.get_model()
print(qmod)

In [25]:

qprog = synthesize(serialized_model)

circuit = GeneratedCircuit.from_qprog(qprog)

print(circuit.data.qubit_mapping.logical_outputs)

{'a': (0, 1), 'b': (2, 3), 'sum': (4, 5, 6)}


In [27]:

show(qprog)

# circuit.save_results("my_arithmetic_circuit.json")

Opening: https://platform.classiq.io/circuit/bf95af1c-0c59-4b49-90e1-c6fe11dff745?version=0.29.2


In [31]:
results_raw = execute(qprog).result()
results = results_raw[0].value
print(results.counts)

{'1011011': 514, '0010100': 469, '0101000': 556, '1000111': 509}


In [32]:
output_results = results.counts_of_multiple_outputs(["a", "b", "sum"])
print(output_results)

{('11', '01', '101'): 514, ('00', '10', '100'): 469, ('00', '01', '010'): 556, ('11', '10', '001'): 509}


In [33]:
def str2num(str):
    return int(str[::-1], 2)

for tupple in output_results.keys():
    print(str2num(tupple[0]), "+", str2num(tupple[1]), "=", str2num(tupple[2]))

3 + 2 = 5
0 + 1 = 1
0 + 2 = 2
3 + 1 = 4


In [34]:
print(
    "circuit width: ",
    circuit.data.width,
    " circuit depth: ",
    circuit.transpiled_circuit.depth,
)

circuit width:  8  circuit depth:  27


In [35]:
# optimising for widht
constraints = Constraints(optimization_parameter="width")
serialized_model_optimized_for_width = set_constraints(
    serialized_model, constraints=constraints
)
qprog_optimized_for_width = synthesize(serialized_model_optimized_for_width)

In [36]:
circuit_optimized_for_width = GeneratedCircuit.from_qprog(qprog_optimized_for_width)

print(
    "circuit width: ",
    circuit_optimized_for_width.data.width,
    " circuit depth: ",
    circuit_optimized_for_width.transpiled_circuit.depth,
)

circuit width:  7  circuit depth:  31


In [41]:
# 
from classiq.builtin_functions import RYGate
import numpy as np


In [43]:
model2 = Model()
ry_params = RYGate(theta='x') # defining building block
model2.RYGate(params = ry_params) # wiring building block into model

#defining sample execution scheme
execution_params = {'x': np.pi/7}
model.sample(execution_params=execution_params)

In [44]:
qprog2 = synthesize(model2.get_model())
results = execute(qprog2)

ClassiqAPIError: Call to API failed with code 500: Error number 11193 occurred. Classical function not defined, and the quantum function requires execution parameters. Please define the classical function.

Error identifier: E5DA94D18-27B4-456C-9547-06B478875334.
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack
If the error persists feel free to open a ticket at: https://short.classiq.io/support